In [1]:
from src.models.SwinUnet.networks.vision_transformer import SwinUnet

class Config:
    class DATA:
        IMG_SIZE = 128 
    
    class MODEL:
        DROP_RATE = 0.0  
        DROP_PATH_RATE = 0.1  
        LABEL_SMOOTHING = 0.1
        NAME = 'swin_tiny_patch4_window7_224'
        PRETRAIN_CKPT = 'src/models/SwinUnet/networks/swin_tiny_patch4_window7_224.pth'

        
        class SWIN:
            PATCH_SIZE = 4 
            IN_CHANS = 35  
            EMBED_DIM = 96  
            DEPTHS = [2, 2, 6, 2]  
            NUM_HEADS = [3, 6, 12, 24] 
            WINDOW_SIZE = 4  
            MLP_RATIO = 4.0  
            QKV_BIAS = True  
            QK_SCALE = None 
            APE = False
            PATCH_NORM = True

    class TRAIN:
        START_EPOCH = 0
        EPOCHS = 300
        WARMUP_EPOCHS = 20
        WEIGHT_DECAY = 0.05
        BASE_LR = 5e-4
        WARMUP_LR = 5e-7
        MIN_LR = 5e-6
        CLIP_GRAD = 5.0
        AUTO_RESUME = True
        ACCUMULATION_STEPS = 0
        USE_CHECKPOINT = False


        class LR_SCHEDULER:
            NAME = 'cosine'
            DECAY_EPOCHS = 30
            DECAY_RATE = 0.1

        class OPTIMIZER:
            NAME = 'adamw'
            EPS = 1e-8
            BETAS = (0.9, 0.999)
            MOMENTUM = 0.9
    
config = Config()

swin_unet = SwinUnet(config, num_classes=1)

/home/sl221120/anaconda3/envs/fire_ts/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


SwinTransformerSys expand initial----depths:[2, 2, 6, 2];depths_decoder:[1, 2, 2, 2];drop_path_rate:0.1;num_classes:1


/home/sl221120/anaconda3/envs/fire_ts/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


---final upsample expand_first---


In [4]:
import torch 

for name, param in swin_unet.named_parameters():
    if param.dtype != torch.float32:
        print(f"Layer: {name}, Precision: {param.dtype}")



In [26]:
import torch

from src.models.SwinUnet.networks.swin_transformer_unet_skip_expand_decoder_sys import PatchEmbed


patch_embedder = PatchEmbed()
inputs = torch.rand(1, 3, 224, 224)
embedding = patch_embedder(inputs)
print(f"when input size is {inputs.shape}, output size is: {embedding.shape}")

new_inputs = torch.rand(1, 3, 128, 128)
new_patch_embedder = PatchEmbed(img_size=128, patch_size=4)
new_embedding = new_patch_embedder(new_inputs)
print(f"when input size is {new_inputs.shape}, output size is: {new_embedding.shape}")




when input size is torch.Size([1, 3, 224, 224]), output size is: torch.Size([1, 3136, 96])
when input size is torch.Size([1, 3, 128, 128]), output size is: torch.Size([1, 1024, 96])


In [17]:
inputs.shape

torch.Size([3, 224, 224])

In [13]:
import torch
import copy
pretrained_path = config.MODEL.PRETRAIN_CKPT

if pretrained_path is not None:
    print("pretrained_path:{}".format(pretrained_path))
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    pretrained_dict = torch.load(pretrained_path, map_location=device)
    pretrained_dict = pretrained_dict['model']
    print("---start load pretrained modle of swin encoder---")
    
    used_keys, unused_keys = [], []

    model_dict = swin_unet.state_dict()
    full_dict = copy.deepcopy(pretrained_dict)
    for k, v in pretrained_dict.items():
        if "layers." in k:
            current_layer_num = 3-int(k[7:8])
            current_k = "layers_up." + str(current_layer_num) + k[8:]
            full_dict.update({current_k:v})
    for k in list(full_dict.keys()):
        if k in model_dict:
            if full_dict[k].shape == model_dict[k].shape:
                used_keys.append(k)
            else:
                if "patch_embed.proj.weight" in k:
                        print(f"Updating patch embedding layer: {k}")
                        pretrained_weight = full_dict[k]
                        new_weight = torch.nn.init.xavier_uniform_(torch.empty_like(model_dict[k]))
                        num_pretrained_channels = pretrained_weight.shape[1]
                        new_weight[:, :num_pretrained_channels] = pretrained_weight[:, :num_pretrained_channels]
                        full_dict[k] = new_weight  # Use updated weights
                        used_keys.append(k)
                print("delete:{};shape pretrain:{};shape model:{}".format(k,v.shape,model_dict[k].shape))
                del full_dict[k]
        else:
            unused_keys.append(k)
    msg = swin_unet.load_state_dict(full_dict, strict=False)
    print("Used pretrained keys:", used_keys)
    print("Unused pretrained keys:", unused_keys)
    print(msg)
else:
    print("none pretrain")


pretrained_path:src/models/SwinUnet/networks/swin_tiny_patch4_window7_224.pth
---start load pretrained modle of swin encoder---
Used pretrained keys: []
Unused pretrained keys: ['patch_embed.proj.weight', 'patch_embed.proj.bias', 'patch_embed.norm.weight', 'patch_embed.norm.bias', 'layers.0.blocks.0.norm1.weight', 'layers.0.blocks.0.norm1.bias', 'layers.0.blocks.0.attn.qkv.weight', 'layers.0.blocks.0.attn.qkv.bias', 'layers.0.blocks.0.attn.proj.weight', 'layers.0.blocks.0.attn.proj.bias', 'layers.0.blocks.0.norm2.weight', 'layers.0.blocks.0.norm2.bias', 'layers.0.blocks.0.mlp.fc1.weight', 'layers.0.blocks.0.mlp.fc1.bias', 'layers.0.blocks.0.mlp.fc2.weight', 'layers.0.blocks.0.mlp.fc2.bias', 'layers.0.blocks.1.norm1.weight', 'layers.0.blocks.1.norm1.bias', 'layers.0.blocks.1.attn.qkv.weight', 'layers.0.blocks.1.attn.qkv.bias', 'layers.0.blocks.1.attn.proj.weight', 'layers.0.blocks.1.attn.proj.bias', 'layers.0.blocks.1.norm2.weight', 'layers.0.blocks.1.norm2.bias', 'layers.0.blocks.1.mlp

In [2]:
model.load_from(config)


pretrained_path:src/models/SwinUnet/networks/swin_tiny_patch4_window7_224.pth
---start load pretrained modle of swin encoder---
delete:patch_embed.proj.weight;shape pretrain:torch.Size([768, 1536]);shape model:torch.Size([96, 35, 4, 4])
delete:layers.0.blocks.0.attn.relative_position_index;shape pretrain:torch.Size([768, 1536]);shape model:torch.Size([16, 16])
delete:layers.0.blocks.1.attn.relative_position_index;shape pretrain:torch.Size([768, 1536]);shape model:torch.Size([16, 16])
delete:layers.1.blocks.0.attn.relative_position_index;shape pretrain:torch.Size([768, 1536]);shape model:torch.Size([16, 16])
delete:layers.1.blocks.1.attn.relative_position_index;shape pretrain:torch.Size([768, 1536]);shape model:torch.Size([16, 16])
delete:layers.2.blocks.0.attn.relative_position_index;shape pretrain:torch.Size([768, 1536]);shape model:torch.Size([16, 16])
delete:layers.2.blocks.1.attn.relative_position_index;shape pretrain:torch.Size([768, 1536]);shape model:torch.Size([16, 16])
delete:

In [2]:
import pandas as pd

def print_avg_test_AP(file_path, model_name):
    df = pd.read_csv(file_path)
    if 'data.features_to_keep' in df.columns:
        stats_ap_per_feature_set = df.groupby('data.features_to_keep')['test_AP'].agg(['mean', 'std'])
        print(model_name)
        print(stats_ap_per_feature_set.applymap(lambda x: f"{x:.3f}"))
    elif "optimizer.lr" in df.columns:
        stats_ap_per_feature_set = df.groupby("optimizer.lr")["test_AP"].agg(["mean", "std"])
        print(model_name)
        print(stats_ap_per_feature_set.applymap(lambda x: f"{x:.3f}"))

    else:
        overall_avg_ap = df['test_AP'].mean()
        overall_std_ap = df['test_AP'].std()
        print(model_name)
        print(f"Overall Average test_AP: {overall_avg_ap:.3f}")
        print(f"Overall Std test_AP: {overall_std_ap:.3f}")

       

In [2]:
file_path = '/home/sl221120/WildfireSpreadTS/results/swin_mono_multi_all.csv'
model_name = "swin_monotemp_multi_all"
print_avg_test_AP(file_path, model_name)


swin_monotemp_multi_all
                                                     mean    std
data.features_to_keep                                           
None                                                0.340  0.094
[0,1,2,3,4,5,6,7,8,9,11,12,13,14,16,17,18,19,20...  0.352  0.088


In [3]:
file_path = '/home/sl221120/WildfireSpreadTS/results/swin_mono_veg.csv'
model_name = "swin_monotemp_veg"
print_avg_test_AP(file_path, model_name)


swin_monotemp_veg
Overall Average test_AP: 0.348
Overall Std test_AP: 0.086


In [4]:
file_path = '/home/sl221120/WildfireSpreadTS/results/swin_multi_veg.csv'
model_name = "swin_multitemp_veg"
print_avg_test_AP(file_path, model_name)


swin_multitemp_veg
Overall Average test_AP: 0.362
Overall Std test_AP: 0.090


In [5]:
file_path = '/home/sl221120/WildfireSpreadTS/results/swin_multi_multi_all.csv'
model_name = "swin_multitemp_multi_all"
print_avg_test_AP(file_path, model_name)

swin_multitemp_multi_all
                                                     mean    std
data.features_to_keep                                           
None                                                0.324  0.097
[0,1,2,3,4,5,6,7,8,9,11,12,13,14,16,17,18,19,20...  0.350  0.086


In [7]:
file_path = '/home/sl221120/WildfireSpreadTS/results/unet_mono_multi_all.csv'
model_name = "unet_monotemp_multi_all"
print_avg_test_AP(file_path, model_name)

unet_monotemp_multi_all
                                                     mean    std
data.features_to_keep                                           
None                                                0.340  0.085
[0,1,2,3,4,5,6,7,8,9,11,12,13,14,16,17,18,19,20...  0.338  0.077


In [3]:
file_path = '/home/sl221120/WildfireSpreadTS/results/unet_multi_multi_all.csv'
model_name = "unet_multitemp_multi_all"
print_avg_test_AP(file_path, model_name)

unet_multitemp_multi_all
                                                     mean    std
data.features_to_keep                                           
None                                                0.337  0.088
[0,1,2,3,4,5,6,7,8,9,11,12,13,14,16,17,18,19,20...  0.350  0.073


In [5]:
file_path = '/home/sl221120/WildfireSpreadTS/results/unet_multi_veg_lrs1.csv'
model_name = "unet_multitemp_veg_lrs_1"
print_avg_test_AP(file_path, model_name)

unet_multitemp_veg_lrs_1
               mean    std
optimizer.lr              
0.001         0.199  0.070
0.010         0.029  0.040


In [4]:
file_path = '/home/sl221120/WildfireSpreadTS/results/unet_multi_veg_lrs2.csv'
model_name = "unet_multitemp_veg_lrs_2"
print_avg_test_AP(file_path, model_name)

unet_multitemp_veg_lrs_2
               mean    std
optimizer.lr              
0.000001      0.277  0.093
0.000010      0.341  0.097


In [1]:
import os
import subprocess

# Define the base directory containing the runs
base_dir = '/home/sl221120/WildfireSpreadTS/lightning_logs/wildfire_progression'
data_dir = '/home/sl221120/scratch/WildfireSpreadTS_HDF5'

# Log data mapping run_id to data_fold_id
log_data = [
    {"run_id":"4adsj08p", "data_fold_id":0},
    {"run_id":"heg3b8sr", "data_fold_id":1},
    {"run_id":"g4kcwn56", "data_fold_id":2},
    {"run_id":"7t9l4h3a", "data_fold_id":3},
    {"run_id":"cgce7y3v", "data_fold_id":4},
    {"run_id":"h495vls8", "data_fold_id":5},
    {"run_id":"e90loozf", "data_fold_id":6},
    {"run_id":"81azfn75", "data_fold_id":7},
    {"run_id":"p8y7pgep", "data_fold_id":8},
    {"run_id":"a1cyprhl", "data_fold_id":9},
    {"run_id":"ju2yiuod", "data_fold_id":10},
    {"run_id":"w09hg1ub", "data_fold_id":11},
]

# Convert log data to a dictionary
run_id_to_fold_id = {entry["run_id"]: entry["data_fold_id"] for entry in log_data}

# Function to run the test command for a specific checkpoint
def run_test(checkpoint_path, data_fold_id):
    test_command = [
        'python',
        '/home/sl221120/WildfireSpreadTS/src/train.py',
        '--config=cfgs/Swin/swin.yaml',
        '--trainer=cfgs/trainer_single_gpu.yaml',
        '--data=cfgs/data_monotemporal_veg_features.yaml',
        '--seed_everything=0',
        #f'--data.data_fold_id={data_fold_id}',
        #f'--data.data_dir={data_dir}'
        f'--ckpt_path={checkpoint_path}'
    ]

    # Run the test command
    subprocess.run(test_command)

# Iterate through each run folder
# for run_id, data_fold_id in run_id_to_fold_id.items():
#     run_path = os.path.join(base_dir, run_id, 'checkpoints')
#     # Find the checkpoint file with the best validation loss
#     checkpoint_file = os.listdir(run_path)[0]
#     if checkpoint_file.endswith(".ckpt"):
#         run_test(checkpoint_file, data_fold_id)


In [2]:
run_id = "k7s96l2o"
features_to_keep = [0, 1, 2, 3, 4, 38, 39]
run_path = os.path.join(base_dir, run_id, 'checkpoints')
checkpoint_file = os.listdir(run_path)[0]
checkpoint_path = (os.path.join(run_path, checkpoint_file))
checkpoint_path


'/home/sl221120/WildfireSpreadTS/lightning_logs/wildfire_progression/k7s96l2o/checkpoints/epoch=122-step=9594.ckpt'

In [3]:
run_test(checkpoint_path=checkpoint_path, data_fold_id=0)

Global seed set to 0


Using the following dataset split:
Train years: [2018, 2019], Val years: [2020], Test years: [2021]


wandb: Currently logged in as: saadlahrichi (bass-connections-22-23). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.18.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.3
wandb: Run data is saved locally in ./lightning_logs/wandb/run-20240913_184208-y3s6va3v
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run models.SwinUnetLightning
wandb: ⭐️ View project at https://wandb.ai/bass-connections-22-23/wildfire_progression
wandb: 🚀 View run at https://wandb.ai/bass-connections-22-23/wildfire_progression/runs/y3s6va3v
/home/sl221120/anaconda3/envs/fire_ts/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


SwinTransformerSys expand initial----depths:[2, 2, 6, 2];depths_decoder:[1, 2, 2, 2];drop_path_rate:0.1;num_classes:1
---final upsample expand_first---


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


...

Restoring states from the checkpoint path at /home/sl221120/WildfireSpreadTS/lightning_logs/wildfire_progression/k7s96l2o/checkpoints/epoch=122-step=9594.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
Loaded model weights from the checkpoint at /home/sl221120/WildfireSpreadTS/lightning_logs/wildfire_progression/k7s96l2o/checkpoints/epoch=122-step=9594.ckpt


Testing DataLoader 0:   0%|          | 0/3337 [00:00<?, ?it/s]

/home/sl221120/anaconda3/envs/fire_ts/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: TorchMetricsUserWarning: You are trying to use a metric in deterministic mode on GPU that uses `torch.cumsum`, which is currently not supported. The tensor will be copied to the CPU memory to compute it and then copied back to GPU. Expect some slowdowns.
  warnings.warn(*args, **kwargs)  # noqa: B028
/home/sl221120/anaconda3/envs/fire_ts/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples found in target, recall is undefined. Setting recall to one for all thresholds.
  warnings.warn(*args, **kwargs)  # noqa: B028


Testing DataLoader 0: 100%|██████████| 3337/3337 [05:15<00:00, 10.58it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         test_AP            0.46942704916000366
         test_f1            0.5620484948158264
        test_iou            0.3908675014972687
        test_loss           0.3750905990600586
     test_precision         0.5122132897377014
       test_recall          0.6226263046264648
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


wandb: Waiting for W&B process to finish... (success).
wandb: 
wandb: Run history:
wandb:               epoch ▁
wandb:             test_AP ▁
wandb:             test_f1 ▁
wandb:            test_iou ▁
wandb:           test_loss ▁
wandb:      test_precision ▁
wandb:         test_recall ▁
wandb: trainer/global_step ▁
wandb: 
wandb: Run summary:
wandb:               epoch 0
wandb:             test_AP 0.46943
wandb:             test_f1 0.56205
wandb:            test_iou 0.39087
wandb:           test_loss 0.37509
wandb:      test_precision 0.51221
wandb:         test_recall 0.62263
wandb: trainer/global_step 0
wandb: 
wandb: 🚀 View run models.SwinUnetLightning at: https://wandb.ai/bass-connections-22-23/wildfire_progression/runs/y3s6va3v
wandb: Synced 6 W&B file(s), 2 media file(s), 3 artifact file(s) and 1 other file(s)
wandb: Find logs at: ./lightning_logs/wandb/run-20240913_184208-y3s6va3v/logs


In [3]:
log_data = [
    {"run_id":"4adsj08p", "data_fold_id":0},
    {"run_id":"heg3b8sr", "data_fold_id":1},
    {"run_id":"g4kcwn56", "data_fold_id":2},
    {"run_id":"7t9l4h3a", "data_fold_id":3},
    {"run_id":"cgce7y3v", "data_fold_id":4},
    {"run_id":"h495vls8", "data_fold_id":5},
    {"run_id":"e90loozf", "data_fold_id":6},
    {"run_id":"81azfn75", "data_fold_id":7},
    {"run_id":"p8y7pgep", "data_fold_id":8},
    {"run_id":"a1cyprhl", "data_fold_id":9},
    {"run_id":"ju2yiuod", "data_fold_id":10},
    {"run_id":"w09hg1ub", "data_fold_id":11},
]

# Convert log data to a dictionary
run_id_to_fold_id = {entry["run_id"]: entry["data_fold_id"] for entry in log_data}


run_id_to_fold_id.items()

dict_items([('4adsj08p', 0), ('heg3b8sr', 1), ('g4kcwn56', 2), ('7t9l4h3a', 3), ('cgce7y3v', 4), ('h495vls8', 5), ('e90loozf', 6), ('81azfn75', 7), ('p8y7pgep', 8), ('a1cyprhl', 9), ('ju2yiuod', 10), ('w09hg1ub', 11)])

In [4]:
import pandas as pd 

def compute_ap(file_path):
    df = pd.read_csv(file_path)

    # Compute the average and standard deviation of test_AP
    test_ap_mean = df['test_AP'].mean()
    test_ap_std = df['test_AP'].std()


    print(test_ap_mean, test_ap_std)


In [2]:
file_path = '/home/sl221120/WildfireSpreadTS/src/veg_mono.csv'
print("veg_mono")
compute_ap(file_path)

veg_mono
0.3228241143127282 0.09496183157932543


In [3]:
file_path = '/home/sl221120/WildfireSpreadTS/src/multi_mono.csv'
print("multi_mono")
compute_ap(file_path)



multi_mono
0.33916346604625386 0.09063526435030687


In [4]:
file_path = '/home/sl221120/WildfireSpreadTS/src/all_mono.csv'
print("all_mono")
compute_ap(file_path)


all_mono
0.32728181034326553 0.10340745934365056


In [6]:
file_path = '/home/sl221120/WildfireSpreadTS/src/veg_multi.csv'
print("veg_multi")
compute_ap(file_path)

veg_multi
0.3335219609240691 0.08486465050508904


In [7]:
file_path = '/home/sl221120/WildfireSpreadTS/src/multi_multi.csv'
print("multi_multi")
compute_ap(file_path)

multi_multi
0.3407648255427678 0.08584146476653304


In [6]:
file_path = '/home/sl221120/WildfireSpreadTS/src/all_multi.csv'
print("all_multi")
compute_ap(file_path)

all_multi
0.29986127962668735 0.10934251644823732


In [7]:
file_path = '/home/sl221120/WildfireSpreadTS/src/all_multi_new.csv'
print("all_multi")
compute_ap(file_path)

all_multi
0.29986127962668735 0.10934251644823731


In [ ]:
print("Veg Mono Unet FoldId0:")

